# SETUP

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine

# GET DATA

In [2]:
# fetch a web page
htlm_page = requests.get("https://www.francetvinfo.fr/sports/")
print(htlm_page.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Sports</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Sports" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Sports&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_FR" /><meta pro

In [3]:
page = BeautifulSoup(htlm_page.text, "html5lib")
print(page.get_text()[:1000])

Sports  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/sports/"+queryString+myHash));  (function(){("und

## EXTRACT TITLES

In [4]:
# extract title
summaries = page.find_all('div', class_="flowItem")

In [5]:
summaries[0]

<div class="flowItem feature" entity="CMS-Article_3829745"><a href="/sports/foot/foot-l-attaquant-de-porto-moussa-marega-quitte-le-terrain-apres-avoir-ete-victime-de-cris-racistes_3829745.html"><figure id="image_3629997"><img alt="L\'attaquant de Porto Moussa Marega, célèbre son but face au Vitoria Guimaraes SC, dimanche 16 février 2020.&amp;nbsp;" botag="CMS-IMAGE_3629997" class="b-lazy" data-src="/image/75runp4ev-2c7c/840/472/20973621.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="L'attaquant de Porto Moussa Marega, célèbre son but face au Vitoria Guimaraes SC, dimanche 16 février 2020.  | MIGUEL RIOPA / AFP" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">Foot : l'attaquant de Porto Moussa Marega quitte le terrain après avoir été victime de cris racistes</h2><div class="description" field="description">Excédé par les chants racistes et les cris de singe qu'il a entendus, il a décidé de qu

In [6]:
summaries[0].select_one("h2").get_text()

"Foot : l'attaquant de Porto Moussa Marega quitte le terrain après avoir été victime de cris racistes"

In [7]:
len(summaries)

29

In [8]:
sport_news = [x.select_one("h2").get_text() for x in summaries]

In [9]:
sport_news

["Foot : l'attaquant de Porto Moussa Marega quitte le terrain après avoir été victime de cris racistes",
 '"Bourré de qualités", "drôle" et "cycliste raté" : qui est Émilien Jacquelin, le champion du monde de poursuite en biathlon',
 "La médaille du jour. Confiné par l'épidémie de Covid-19, ce marathonien chinois court... autour de son lit",
 'Biathlon : le Français Emilien Jacquelin sacré champion du monde de la poursuite',
 'VIDEO. "Au moment de ma maladie, je haïssais mon corps. Aujourd\'hui, je vis ma vie à 100%" : Théo, champion de natation amputé des mains et des pieds à six ans',
 'Tests de grossesse dans le handball : six joueuses contestent la version de leur club nantais',
 "Handball : ce que l'on sait sur l'affaire des tests de grossesse qui secoue un club nantais",
 'NBA : le trophée récompensant le meilleur basketteur du All-Star Game rebaptisé en hommage à Kobe Bryant',
 'En route vers Paris 2024. Le sport mène à tout',
 "Ligue 1 : au terme d'un match fou, le Paris Saint-

# WRITE THE DATA IN POSTGRESQL DB

In [10]:
df = pd.DataFrame({"source": "franceinfo", "headline": sport_news, "date": pd.to_datetime('today').to_pydatetime().date()})

In [11]:
df.head()

,source,headline,date
0,franceinfo,Foot : l'attaquant de Porto Moussa Marega quit...,2020-02-17
1,franceinfo,"""Bourré de qualités"", ""drôle"" et ""cycliste rat...",2020-02-17
2,franceinfo,La médaille du jour. Confiné par l'épidémie de...,2020-02-17
3,franceinfo,Biathlon : le Français Emilien Jacquelin sacré...,2020-02-17
4,franceinfo,"VIDEO. ""Au moment de ma maladie, je haïssais m...",2020-02-17


In [12]:
# create an engine
alchemy_engine = create_engine('postgresql+psycopg2://antoinemertz@localhost/news', pool_recycle=3600)

In [14]:
conn = alchemy_engine.connect()
table = "sport"

In [15]:
try:
    frame = df.to_sql(name=table, con=conn, if_exists='append', index=False)
except Exception as ex:
    print(ex)
conn.close()